In [1]:

import os
import torch
import clip
from PIL import Image


device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
model, preprocess = clip.load("ViT-B/32", device=device)

data_path = "/home/s0001396/Documents/phd/datasets/once/data/"
sequences = os.listdir(data_path)
image_feat = dict()
for seq in sequences:
    seq_path = os.path.join(data_path, seq, "cam01")
    imgs = os.listdir(seq_path)
    for img in imgs:
        image_path = os.path.join(seq_path, img)
        image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
        with torch.no_grad():
            image_features = model.encode_image(image)
            image_feat[img] = image_features



cuda


KeyboardInterrupt: 

In [4]:
text = clip.tokenize(["a car on the left"]).to(device)

with torch.no_grad():
    text_features = model.encode_text(text)

In [5]:
def logit_img_txt(img_feat, txt_feat, model):
    img_feat = img_feat / img_feat.norm(dim=1, keepdim=True)
    txt_feat = txt_feat / txt_feat.norm(dim=1, keepdim=True)

    # cosine similarity as logits
    logit_scale = model.logit_scale.exp()
    logits_per_image = logit_scale * image_features @ text_features.t()
    logits_per_text = logits_per_image.t()
    return logits_per_text

In [6]:
tmp = [logit_img_txt(img_feat, text_features, model) for img, img_feat in image_feat.items()]

In [9]:
tmp[2]

(tensor([[2870.]], device='cuda:0', dtype=torch.float16, grad_fn=<MmBackward0>),
 tensor([[2870.]], device='cuda:0', dtype=torch.float16, grad_fn=<TBackward0>))